# 2 - Entrenar la red neuronal para reconocer letras

Luego de que tenemos las imagenes listas, lo que nos resta es entrenar nuestra red neuronal para que pueda aprender cada una de las 32 letras/números posibles. 

In [ ]:
import cv2
import pickle
import os.path
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
from helpers import resize_to_fit

# Probar el modelo
from keras.models import load_model
import random

import matplotlib.pyplot as plt

%matplotlib inline

LETTER_IMAGES_FOLDER = "extracted_letter_images"
MODEL_FILENAME = "captcha_model.hdf5"
MODEL_LABELS_FILENAME = "model_labels.dat"

In [ ]:
# inicializar los datos y las etiquetas
data = []
labels = []

# Revisar cada imagen de entrada
for image_file in paths.list_images(LETTER_IMAGES_FOLDER):    
    # Cargue la imagen y conviértala a escala de grises
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cambiar el tamaño de la letra para que quepa en un cuadro de 20x20 píxeles
    image = resize_to_fit(image, 20, 20)
    #print("Imagen Ampliada de 2 dimensiones: " + str(image.shape))
    
    # Agregue una tercera dimensión de canal a la imagen para hacer feliz a Keras
    image = np.expand_dims(image, axis=2)
    #print("Image 3 channel dimension: " + str(image.shape))
        
    # Tome el nombre de la letra en función de la carpeta en la que estaba
    label = image_file.split(os.path.sep)[-2]

    # Agregue la imagen de la letra y su etiqueta a nuestros datos de entrenamiento
    data.append(image)
    labels.append(label)

Las imagenes son representadas en arreglos, numeros de 0 al 255. 

In [ ]:
print("Data: " + str(data[23323]))
print("Labels: " + str(labels[23323]))

Transformaremos la representación del arreglo en una representación decimal de 0 a 1 para mejorar el entrenamiento.

In [ ]:
print(24/255.0)

In [ ]:
# Escala las intensidades de píxeles sin procesar al rango [0, 1] (esto mejora el entrenamiento)
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)


In [ ]:
print("Data: " + str(data[23323]))
print("Labels: " + str(labels[23323]))

Ahora tenemos los datasets de entrenamientos listos, las imagenes y el resultado, basicamente podemos asociar que la posición 
23323 corresponde a la letra M.

Utilizaremos **X_train** y **X_test** como los conjuntos de datos con lás **imágenes de entrenamiento y validación respectivamente**, mientras que **Y_train** y **Y_test** son los datasets con **las etiquetas**. 

In [ ]:
# Dividir los datos de entrenamiento en trenes y conjuntos de pruebas separados
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, random_state=0)

# Convierta las etiquetas (letras) en codificaciones únicas con las que Keras puede trabajar
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

# Tamaño de la data
print("Tamaño de la data: " + str(len(data)))

# Tamaño de data de entrenamiento
print("Tamaño de data de entrenamiento: " + str(len(Y_train)))

# Representación binaria del resultado
print(Y_train[23])

# Quitar la dimensión que habíamos puesto para imprimir la imagen
result = X_train[23][:, :, 0]

print(result.shape)
plt.imshow(result, cmap = plt.cm.binary)

La palabra **convolución** suena como un término elegante y complicado, pero en realidad no lo es. De hecho, si alguna vez ha trabajado con diseño gráfico, procesamiento de imágenes u OpenCV antes, ¡ya ha aplicado convoluciones, lo sepas o no!

¿Alguna vez has aplicado desenfoque o suavizado? Sí, eso es una convolución.

¿Qué pasa con la detección de bordes? Sí, convolución.

¿Has abierto Photoshop o GIMP para enfocar una imagen? Lo has adivinado: convolución.

Las convoluciones son uno de los componentes básicos más críticos y fundamentales en la visión por computadora y el procesamiento de imágenes.

https://skymind.ai/wiki/convolutional-network

## Convolución y pooling

En el ejemplo anterior sobre los dígitos escritos a mano, usamos parches de imagen de 8×8 píxeles para entrenar un auto-codificador. La pregunta es, ¿por qué no introducir las imágenes completas?

Cuanto mayor es la entrada del auto-codificador, más pesos hay que entrenar, y más lento es todo el proceso de entrenamiento. Si tenemos imágenes de 1000×1000 pixeles, y utilizamos 1000 neuronas en la capa oculta, necesitaríamos entrenar 2000 millones de pesos, mientras que si utilizamos parches de 8×8 y utilizamos 100 neuronas en la capa oculta, entrenamos sólo 12800 pesos.
Si usamos imágenes enteras, para el auto-codificador una imagen que tenga un rostro centrado, y otra que tenga el mismo rostro ligeramente desplazado hacia un lado, son completamente diferentes.
Especialmente con el fin de resolver el segundo punto, se puede utilizar una técnica conocida como convolución, y que está basada en cómo se estructuran realmente las neuronas en nuestro sistema visual.

La idea de base es, ¿qué más da dónde aparezca un círculo? Un círculo sigue siendo un círculo aunque lo desplacemos.

Por eso, podemos entrenar un auto-codificador con parches de imagen, y luego desplazar el codificador por toda la imagen, como si fuese un escáner, buscando características. Este proceso transforma la matriz de píxeles en otra matriz de características (listas de números) producidas por el codificador. Es esencialmente la misma imagen, pero cada pixel es mucho más rico en información, y contiene información de la región en que se encuentra el pixel, no sólo del pixel aislado.

![title](img/convolution.png)

El siguiente paso, llamado pooling, consiste en agrupar las características (listas de números) de varias coordinadas contiguas de la imagen, con alguna función de agrupación (la media, o el máximo). Esta etapa reduce la resolución de la imagen. A continuación podemos continuar haciendo convolución y pooling hasta que nos quede una imagen de 1×1 pixeles con muchísima información, o podemos incluir auto-codificadores intermedios que procesen los datos para buscar características de más alto nivel.

![title](img/pooling.png)

En una sola etapa de convolución y pooling, obtenemos una imagen de menos resolución, y que en cada píxel nos cuenta la siguiente historia: “en esta región de la imagen hay un círculo y una línea”. No nos dice exactamente dónde estaba el círculo en la matriz con la resolución original. Si ahora acoplamos otro auto-codificador, alguna neurona puede aprender a activarse siempre que reciba esta información agregada acerca de un círculo y una línea, y cuando apliquemos todo el sistema a nuestros números, esa neurona se activará cuando la imagen tenga dibujado el número nueve, por ejemplo.

https://rubenlopezg.wordpress.com/2014/05/07/que-es-y-como-funciona-deep-learning/

## Construyendo y entrenando la red neuronal

Dado que solo necesitamos reconocer imágenes de letras y números individuales, no necesitamos una arquitectura de red neuronal muy compleja. Reconocer letras es un problema mucho más fácil que reconocer imágenes complejas como imágenes como gatos y perros.

Utilizaremos una arquitectura de red neuronal convolucional simple con dos capas convolucionales y dos capas completamente conectadas:

![title](img/cnn-architecture.png)

## Prueba de buen entrenamiento vs mal entrenamiento

Antes de utilizar la red neuronal convolucional completa, haremos algunos ejercicios para poder entender cual sería el resultado del proceso de aprendizaje de nuestra red neuronal convolucional, cuando tenga un buen entrenamiento vs un mal entrenamiento. 

In [ ]:
# Guarde el mapeo de las etiquetas a codificaciones de un solo hot.
# Necesitaremos esto más adelante cuando usemos el modelo para decodificar lo que significan sus predicciones
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

# ¡Construye la red neuronal!
model = Sequential()

# Primera capa convolucional con agrupación máxima
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))

Hemos creado nuestro primel modelo, en este escenario solamente tenemos la primera capa convolucional, veamos la forma en que esta capa transforma las imagenes.

In [ ]:
# Ask the neural network to make a prediction
pred = model.predict(X_test)

print(pred.shape)

#For all the filters, plot the output of the input
plt.figure(figsize=(18,18))
filts = pred[0]
for i in range(20):
    filter_digit = filts[:,:,i]
    plt.subplot(6,6,i+1)
    plt.imshow(filter_digit,cmap='gray'); plt.axis('off');


In [ ]:
# Guarde el mapeo de las etiquetas a codificaciones de un solo hot.
# Necesitaremos esto más adelante cuando usemos el modelo para decodificar lo que significan sus predicciones
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

# ¡Construye la red neuronal!
model = Sequential()

# Primera capa convolucional con agrupación máxima
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set weights for new model from weights trained on MNIST.
for i in range(1):
    model.layers[i].set_weights(model.layers[i].get_weights())

# Ask the neural network to make a prediction
pred = model.predict(X_test)

print(pred.shape)

#For all the filters, plot the output of the input
plt.figure(figsize=(18,18))
filts = pred[0]
for i in range(20):
    filter_digit = filts[:,:,i]
    plt.subplot(6,6,i+1)
    plt.imshow(filter_digit,cmap='gray'); plt.axis('off');


In [ ]:
# Guarde el mapeo de las etiquetas a codificaciones de un solo hot.
# Necesitaremos esto más adelante cuando usemos el modelo para decodificar lo que significan sus predicciones
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

# ¡Construye la red neuronal!
model = Sequential()

# Primera capa convolucional con agrupación máxima
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Segunda capa convolucional con agrupación máxima
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#set weights for new model from weights trained on MNIST.
for i in range(1):
    model.layers[i].set_weights(model.layers[i].get_weights())

# Ask the neural network to make a prediction
pred = model.predict(X_test)

print(pred.shape)

#For all the filters, plot the output of the input
plt.figure(figsize=(18,18))
filts = pred[0]
for i in range(20):
    filter_digit = filts[:,:,i]
    plt.subplot(6,6,i+1)
    plt.imshow(filter_digit,cmap='gray'); plt.axis('off');


Con estas imagenes tenemos una idea de la forma en que la red neuronal convolucional procesa la información y empieza a aprender de los patrones de las imagenes desde diferentes aspectos. 

Volvamos al inicio y entrenemos nuestro modelo solamente utilizando la primera capa convolucional, a esta muestra llamaremos **Data de entrenamiento mala**

Probaremos qué pasaría si entrenamos nuestra red neuronal convolucional con esta data, ¿cual es su potencial de descubrir las imagenes y el texto?

![title](img/cnn-arch-bad.png)

In [ ]:
# Guarde el mapeo de las etiquetas a codificaciones de un solo hot.
# Necesitaremos esto más adelante cuando usemos el modelo para decodificar lo que significan sus predicciones
with open("bad_model.hdf5", "wb") as f:
    pickle.dump(lb, f)

# ¡Construye la red neuronal!
model = Sequential()

# Primera capa convolucional con agrupación máxima
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))

# Capa oculta con 500 nodos
model.add(Flatten())
model.add(Dense(500, activation="relu"))

# Capa de salida con 32 nodos (uno para cada posible letra / número que predecimos)
model.add(Dense(32, activation="softmax"))

# Pídale a Keras que construya el modelo TensorFlow detrás de escena
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


In [ ]:
# Entrena la red neuronal
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1, verbose=1)

# Guardar el modelo entrenado en el disco
model.save("bad_model.hdf5")

In [ ]:
model = load_model("bad_model.hdf5")

with open(MODEL_LABELS_FILENAME, "rb") as f:
    lb = pickle.load(f)

# Bad DATA:  1269, 719 3042 1175
num = 3042 #random.randint(1,9686)
image = X_test[num][:, :, 0]
image2 = X_test[num]

prediction = model.predict(image2.reshape(1,20,20,1))
    
print('Image Number: ' + str(num))

letter = lb.inverse_transform(prediction)[0]

print(letter)

plt.figure(figsize=(15,4));
plt.subplot(1,2,1);
plt.title('Example of digit: {}'.format(letter));
plt.imshow(image,cmap='gray'); plt.axis('off');
probs = model.predict_proba(image2.reshape(1,20,20,1),batch_size=1)
plt.subplot(1,2,2);
plt.title('Probabilities for each digit class');
plt.bar(np.arange(32),probs.reshape(32),align='center'); 
plt.xticks(np.arange(32),('2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z'));

![title](img/cnn-architecture.png)

In [ ]:
# Guarde el mapeo de las etiquetas a codificaciones de un solo hot.
# Necesitaremos esto más adelante cuando usemos el modelo para decodificar lo que significan sus predicciones
with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

# ¡Construye la red neuronal!
model = Sequential()

# Primera capa convolucional con agrupación máxima
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Segunda capa convolucional con agrupación máxima
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Capa oculta con 500 nodos
model.add(Flatten())
model.add(Dense(500, activation="relu"))

# Capa de salida con 32 nodos (uno para cada posible letra / número que predecimos)
model.add(Dense(32, activation="softmax"))

# Pídale a Keras que construya el modelo TensorFlow detrás de escena
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()


In [ ]:
# Entrena la red neuronal
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=10, verbose=1)

# Epoch - https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

# Guardar el modelo entrenado en el disco
model.save(MODEL_FILENAME)

In [ ]:
model = load_model(MODEL_FILENAME)

with open(MODEL_LABELS_FILENAME, "rb") as f:
    lb = pickle.load(f)


num = 3042 #,5940,2433,5936,9005
image = X_test[num][:, :, 0]
image2 = X_test[num]

prediction = model.predict(image2.reshape(1,20,20,1))
    
print(image.shape)

letter = lb.inverse_transform(prediction)[0]

print(letter)

plt.figure(figsize=(15,4));
plt.subplot(1,2,1);
plt.title('Example of digit: {}'.format(letter));
plt.imshow(image,cmap='gray'); plt.axis('off');
probs = model.predict_proba(image2.reshape(1,20,20,1),batch_size=1)
plt.subplot(1,2,2);
plt.title('Probabilities for each digit class');
plt.bar(np.arange(32),probs.reshape(32),align='center'); 
plt.xticks(np.arange(32),('2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z'));